In [1]:
%fs ls dbfs:/FileStore/tables/

path,name,size
dbfs:/FileStore/tables/FileStore/,FileStore/,0
dbfs:/FileStore/tables/airquality.csv,airquality.csv,3869
dbfs:/FileStore/tables/dataJun_1_2020__10_-967d1.csv,dataJun_1_2020__10_-967d1.csv,22610
dbfs:/FileStore/tables/dataJun_1_2020__11_-d18a9.csv,dataJun_1_2020__11_-d18a9.csv,22440
dbfs:/FileStore/tables/dataJun_1_2020__12_-43eb8.csv,dataJun_1_2020__12_-43eb8.csv,22563
dbfs:/FileStore/tables/dataJun_1_2020__13_-4aafe.csv,dataJun_1_2020__13_-4aafe.csv,22489
dbfs:/FileStore/tables/dataJun_1_2020__2_-38473.csv,dataJun_1_2020__2_-38473.csv,22593
dbfs:/FileStore/tables/dataJun_1_2020__3_-5d649.csv,dataJun_1_2020__3_-5d649.csv,22634
dbfs:/FileStore/tables/dataJun_1_2020__4_-ae5f9.csv,dataJun_1_2020__4_-ae5f9.csv,22577
dbfs:/FileStore/tables/dataJun_1_2020__5_-0c5d7.csv,dataJun_1_2020__5_-0c5d7.csv,22634


In [2]:
%fs head dbfs:/FileStore/tables/dataJun_1_2020__10_-967d1.csv --maxBytes=492

[Truncated to first 492 bytes]
Cust_Name|IMEI|IMSI|Contact|Company|GUID|Credit_card_track1|Credit_card_track2|Credit_card_PAN|CVV|Transaction_Date
Hines, Jayme O.|16200206 7615|82489730799|(007) 86358383|Sit Amet Lorem Company|4FAD6BA0-49DA-C51B-3AD4-B53910AEEFAB|%B311251982586656^MichelleLandry^35051114490?9|;340630864714038=14051142211? |491786 864271 9029|795|10/12/2019
Francis, Quinlan M.|16491028 5958|21924303899|(036680) 371766|Mi Enim Condimentum Institute|CAB6F074-869D-B67E-6DB1-3F14E84852D8|%B589384593440^H

In [3]:
customerMonth1DF = (spark
  .read                                                # Call the read method returning a DataFrame
  .option("inferSchema","true")                        # Option to tell Spark to infer the schema
  .option("header","true")                             # Option telling Spark that the file has a header
  .option("delimiter", '|')
  .csv("/FileStore/tables/dataJun_1_2020__2_-38473.csv"))  # Option telling Spark where the file is

In [4]:
audit_input = customerMonth1DF[['Cust_Name','IMEI','IMSI','Contact','Company','Credit_card_PAN','Transaction_Date']]

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import os
def main():
    os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk'
    spark=SparkSession.builder.master("local").getOrCreate()
#     df=spark.read.csv('/FileStore/tables/dataJun_1_2020__2_-38473.csv',header=True)
    df = audit_input
    print ("Input Data: \n")
    df.show(2)
    mask_func_udf = udf(mask_func, StringType())
    df_masked=df.withColumn("CardNumber_masked",mask_func_udf(df["Credit_card_PAN"]))
    df_masked=df_masked.drop("Credit_card_PAN").withColumnRenamed("CardNumber_masked","Credit_card_PAN")
    print ("Masked Data: \n")
    df_masked.show(2)
    return 0
def mask_func(colVal):
    if len(colVal)>12:
        charList=list(colVal)
        charList[4:len(colVal)]='x'*8
        return "".join(charList)
    else:
        return colVal
if __name__ == '__main__':
    main()

Input Data: 

+-----------------+-------------+-----------+---------------+--------------------+------------------+----------------+
 Cust_Name| IMEI| IMSI| Contact| Company| Credit_card_PAN|Transaction_Date|
+-----------------+-------------+-----------+---------------+--------------------+------------------+----------------+
Williams, Beck D.|16890818 5864|50407629799|(00780) 3366032| Amet Limited|491709 597686 8277| 23/04/2019|
 Wagner, Thane P.|16800701 5384|24336923099|(031658) 121271|Pellentesque Tinc...| 4929924759331| 23/04/2019|
+-----------------+-------------+-----------+---------------+--------------------+------------------+----------------+
only showing top 2 rows

Masked Data: 

+-----------------+-------------+-----------+---------------+--------------------+----------------+---------------+
 Cust_Name| IMEI| IMSI| Contact| Company|Transaction_Date|Credit_card_PAN|
+-----------------+-------------+-----------+---------------+--------------------+----------------+---------------+
Williams, Beck D.|16890818 5864|50407629799|(00780) 3366032| Amet Limited| 23/04/2019| 4917xxxxxxxx|
 Wagner, Thane P.|16800701 5384|24336923099|(031658) 121271|Pellentesque Tinc...| 23/04/2019| 4929xxxxxxxx|
+-----------------+-------------+-----------+---------------+--------------------+----------------+---------------+
only showing top 2 rows

In [6]:
import pyffx
import secrets
from base64 import urlsafe_b64encode as b64e, urlsafe_b64decode as b64d
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from pip._internal.utils import logging

backend = default_backend()
iterations = 100_000

def _derive_key(password: bytes, salt: bytes, iterations: int = iterations) -> bytes:
    """Derive a secret key from a given password and salt"""
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=iterations,
        backend=backend,
    )
    return b64e(kdf.derive(password))


def encrypt(password: str, message: str, iterations: int = iterations) -> bytes:
    salt = secrets.token_bytes(16)
    key = _derive_key(str(password).encode(), salt, iterations)
    return b64e(
        b"%b%b%b"
        % (
            salt,
            iterations.to_bytes(4, "big"),
            b64d(Fernet(key).encrypt(message.encode())),
        )
    )



token = Fernet.generate_key().decode()

#encrypt(token, message: str, iterations: int = iterations)

z = encrypt(token, '(00780) 3366032')
print("(00780) 3366032")
print('\n')
print('Encrypted Contact Detail')
print(z)
# e = pyffx.Integer(z, length=4)
# y = decrypt(z,token)
# print('\n')
# print(y)
# a = e.encrypt(1234)
#print(a)

# d = e.decrypt(a)
#print(d)

b'Jla3PMR_uIn4msZwBLvVfQABhqCAAAAAAF7eYjbQBh6Mt7h0Y7e4lXyeEVDUsk_EqKWPRvqvEKv8uIuk31m2LJlm0UOBGlZSr8aGS5CFYbche3LQPllzGKrqBCdo'

In [7]:
# Decryption logic

def decrypt(password: str, token: bytes) -> bytes:
    decoded = b64d(token)
    salt, iter, token = decoded[:16], decoded[16:20], b64e(decoded[20:])
    iterations = int.from_bytes(iter, "big")
    key = _derive_key(str(password).encode(), salt, iterations)
    return Fernet(key).decrypt(token).decode()

y = decrypt(token, z)

print(y)

(00780) 3366032